# Analyse ECAD Daten mit Spark (4 Stationen, Regen)

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import pandas as pd
import glob, os

#edac_raw = '../data/ecad/RR_SOUID234717.txt'

Da es nicht möglich ist, mit SparkSession Headerzeilen zu skippen, lesen wir die Daten zuerst als Panda ein, entfernen die unnötigen Zeilen und konvertieren dann in ein Spark-Dataframe.

Im ersten Schritt wurden die Regendaten der Stationen Krems, Poysdorf-Ost, Neusiedl/See sowie Güssing heruntergeladen.
Details zu den Daten (StationenIDs, Infos von der Ecad, Jahre etc. folgen noch....)

In [16]:
#path = r'/home/jovyan/Klimadaten/data/ecad' # use your path
#all_files = glob.glob(os.path.join(path , "/*.txt"))
all_files = ["../data/ecad/RR_SOUID234717.txt","../data/ecad/RR_SOUID234938.txt","../data/ecad/RR_SOUID236251.txt","../data/ecad/RR_SOUID235003.txt"]
all_files    

['../data/ecad/RR_SOUID234717.txt',
 '../data/ecad/RR_SOUID234938.txt',
 '../data/ecad/RR_SOUID236251.txt',
 '../data/ecad/RR_SOUID235003.txt']

Speichern aller Daten in einem Panda-Dataframe.

In [17]:
li = []

for filename in all_files:
    df = pd.read_csv(filename
                     , index_col=None
                     , header=0
                     , sep=","
                   , skiprows=18)
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)
data

,STAID,SOUID,DATE,RR,Q_RR
0,24702,234717,19360101,-9999,9
1,24702,234717,19360102,1,0
2,24702,234717,19360103,3,0
3,24702,234717,19360104,44,0
4,24702,234717,19360105,11,0
...,...,...,...,...,...
110843,24724,235003,20220426,-9999,9
110844,24724,235003,20220427,-9999,9
110845,24724,235003,20220428,-9999,9
110846,24724,235003,20220429,-9999,9


In [18]:
# edac ######### Krems Regen
#data = pd.read_csv(edac_raw
#                   , sep=","
#                   , skiprows=18
#                   , header=0)
#data.head(5)
#len(data)

Einlesen der Panda-Dataframes mit Spark:

In [19]:
spark = SparkSession \
    .builder \
    .appName("Read Data") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(data) 
sparkDF.printSchema()
sparkDF.show()

root
 |--  STAID: long (nullable = true)
 |--     SOUID: long (nullable = true)
 |--     DATE: long (nullable = true)
 |--    RR: long (nullable = true)
 |--  Q_RR: long (nullable = true)

+------+---------+--------+-----+-----+
| STAID|    SOUID|    DATE|   RR| Q_RR|
+------+---------+--------+-----+-----+
| 24702|   234717|19360101|-9999|    9|
| 24702|   234717|19360102|    1|    0|
| 24702|   234717|19360103|    3|    0|
| 24702|   234717|19360104|   44|    0|
| 24702|   234717|19360105|   11|    0|
| 24702|   234717|19360106|    0|    0|
| 24702|   234717|19360107|    0|    0|
| 24702|   234717|19360108|    0|    0|
| 24702|   234717|19360109|    1|    0|
| 24702|   234717|19360110|   53|    0|
| 24702|   234717|19360111|   51|    0|
| 24702|   234717|19360112|   13|    0|
| 24702|   234717|19360113|    0|    0|
| 24702|   234717|19360114|    0|    0|
| 24702|   234717|19360115|    0|    0|
| 24702|   234717|19360116|    0|    0|
| 24702|   234717|19360117|    0|    0|
| 24702|   

In [20]:
sparkDF.schema.names

[' STAID', '    SOUID', '    DATE', '   RR', ' Q_RR']

Bei Ansicht des Schemas sieht man, dass die Spalten mit Leerzeichen beginnen, deshlab speichern wir diese um:

In [22]:
column_names=['STAID', 'SOUID', 'DATE', 'RR', 'Q_RR']
sparkDF = sparkDF.toDF(*column_names)
sparkDF.schema.names

['STAID', 'SOUID', 'DATE', 'RR', 'Q_RR']

Ausgabe der Anzahl der Datensätze / Station:

In [23]:
sparkDF.groupby('SOUID').count().show()

+------+-----+
| SOUID|count|
+------+-----+
|234717|31532|
|234938|31532|
|236251|27149|
|235003|20635|
+------+-----+



Da in den Metadaten die Info zu finden war, dass Datensätze in der Spalte "Q_RR" ungültige Daten enthalten, entfernen wir diese Zeilen:

In [24]:
sparkDF = sparkDF.filter(col("Q_RR") == '0')

Man sieht nun, dass einige Datensätze entfrent wurden:

In [25]:
sparkDF.groupby('SOUID').count().show()

+------+-----+
| SOUID|count|
+------+-----+
|234717|31411|
|234938|30334|
|236251|17655|
|235003|20515|
+------+-----+



In [26]:
sparkDF.printSchema()

root
 |-- STAID: long (nullable = true)
 |-- SOUID: long (nullable = true)
 |-- DATE: long (nullable = true)
 |-- RR: long (nullable = true)
 |-- Q_RR: long (nullable = true)



Nun noch die unnötigen Spalten entfernen: es stehen nun nur noch die interessierenden Werte Station, datum und Regenmenge zur Verfügung:

In [27]:
sparkDF = sparkDF.drop("STAID", "Q_RR")

In [28]:
sparkDF.printSchema()

root
 |-- SOUID: long (nullable = true)
 |-- DATE: long (nullable = true)
 |-- RR: long (nullable = true)

